In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
import glob

## Function Read CSV files of Throughput from Iperf log


In [2]:
def getDataframeThru(df,start_row,measurement_interval,header_range):
    '''
    This functions will import the data from txt file and return the dataframe without the header of txt file.
    Input: 
        measurement_interval = 30 (sec) : 
        header_range = 10 lines
        start_row = 0 
    Return: 
        df1t : dataframe of througput and jitter
    '''
    
    df1 = df.drop(labels=range(start_row, header_range), axis=0)
    df1t = df1.drop(labels=range(measurement_interval, len(df)), axis=0)
    
    return df1t

In [6]:
def readCSV2pd_Thru(directoryPath,tf_load,edge_name):
    """
    This function is to read a CSV file and return the average value and varience of Thruoughput
    input: directoryPath : path of file names
            tf_load : list of traffic load
            
    """
    avg_thru = []
    var_thru = []
    for tf in tf_load:
        df = pd.DataFrame()
        print('TF',tf)
        for file_name in glob.glob(directoryPath+edge_name+str(tf)+'.csv'):
            x = pd.read_csv(file_name,skip_blank_lines=True,header=None)  
            print('x =', x)
            measurement_interval = 60
            start_row = 0
            header_range = 1
            df =getDataframeThru(x,start_row,measurement_interval,header_range)
            df.rename({0 :'Interval', 1 : 'Unit', 2 :'Transfer', 3 :'Unit Byte', 4 :'Bitrate', 5: 'Unit bps', 6:'Total Datagrams'}, axis=1, inplace=True)

            df['Bitrate'] = pd.to_numeric(df['Bitrate'], errors='coerce',downcast="float")
            Throughput = []
            for i in range(1,df['Unit bps'].shape[0]):
                if df['Unit bps'].loc[i] == 'Kbits/sec':
                    Throughput.append(df['Bitrate'].loc[i]*1e-3)
                elif df['Unit bps'].loc[i] == 'Mbits/sec':
                    Throughput.append(df['Bitrate'].loc[i])
                else:
                     pass   
            Thr = np.asarray(Throughput)
            avg_thru.append(Thr.mean())
        
            
    return avg_thru


## Read File CSV

In [7]:


directoryPath = '/Users/kalika/PycharmProjects/Privacy_SDN_Edge_IoT/PlanB/CPU_utilization_Experiment/version3_Experiment_style/Experiment1/Edge1_iperf_log/'



In [8]:
edge_name = 'edge1_M'
tf_load  = [i*2 for i in range(1,20)]
print(tf_load)
avg_thru_edge1 = readCSV2pd_Thru(directoryPath_iperf,tf_load,edge_name)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]
TF 2
TF 4
TF 6
TF 8
TF 10
TF 12
TF 14
TF 16
TF 18
TF 20
TF 22
TF 24
TF 26
TF 28
TF 30
TF 32
TF 34
TF 36
TF 38
